## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [ ]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Mahibadhoo,MV,2022-03-03 02:50:53,3.7833,72.9667,82.87,64,100,13.22
3,3,Tautira,PF,2022-03-03 02:50:53,-17.7333,-149.1500,86.02,76,28,14.52
6,6,Geraldton,AU,2022-03-03 02:50:54,-28.7667,114.6000,75.61,31,0,20.71
10,10,Vanimo,PG,2022-03-03 02:50:56,-2.6741,141.3028,82.20,81,96,2.82
13,13,New Norfolk,AU,2022-03-03 02:50:57,-42.7826,147.0587,83.66,37,12,3.00
14,14,Arraial Do Cabo,BR,2022-03-03 02:50:57,-22.9661,-42.0278,76.78,88,2,20.92
16,16,Rikitea,PF,2022-03-03 02:50:58,-23.1203,-134.9692,80.06,69,11,17.56
17,17,Kapaa,US,2022-03-03 02:50:58,22.0752,-159.3190,80.58,74,40,6.91
21,21,Atuona,PF,2022-03-03 02:51:00,-9.8000,-139.0333,78.53,80,13,20.13
23,23,Teknaf,BD,2022-03-03 02:51:01,20.8624,92.3058,79.03,29,3,6.58


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       154
City          154
Country       154
Date          154
Lat           154
Lng           154
Max Temp      154
Humidity      154
Cloudiness    154
Wind Speed    154
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

KeyError: "['Current Weather'] not in index"

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
For index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [ ]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

clean_hotel_df.head()

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Weather} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure

fig = gmaps.figure(center=(35.0, 32.0), zoom_level=1.8)
fig.add_layer(marker_layer)
fig